In [2]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
# Loading dataset
dataset_dir_train = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/train"
dataset_dir_test = "../input/stanford-car-dataset-by-classes-folder/car_data/car_data/test"

train_transform = transforms.Compose([transforms.Resize((300, 300)),
                                 transforms.RandomRotation(15),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
test_transform = transforms.Compose([transforms.Resize((300, 300)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

dataset = torchvision.datasets.ImageFolder(root=dataset_dir_train, transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir_test, transform = test_transform)
testloader = torch.utils.data.DataLoader(dataset2, batch_size = 32, shuffle=False, num_workers = 2)

In [6]:
# function to train the model
def TrainModel(model, lossFun, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    
    model.train()
    
    for epoch in range(n_epochs):
        since = time.time() # record the start time
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            # Asigning GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            outputs = model(inputs)
            m = nn.LogSoftmax(dim=1)
            _, predicted = torch.max(outputs.data, 1)
            loss = lossFun(m(outputs), labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        timeSpent = time.time()-since # time used
        epoch_loss = running_loss/counter
        print(running_correct, counter)
        epoch_acc = 100*(running_correct/counter)
        print("Epoch %s, Time Used: %d s, loss: %.4f, acc: %.4f" % (epoch+1, timeSpent, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        scheduler.step(epoch_acc)
    print('Training Finished')
    return model, losses, accuracies

# function to evaluate a model
def EvaluateModel(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            # assigning to GPU
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    test_acc = 100.0 * (correct / total)
    print('The accuracy on the test dataset: %d %%' % (
        test_acc))
    return test_acc

In [10]:
from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

<weakproxy at 0x7fcc7c09d770 to Device at 0x7fcc7ea16610>

### Densenet161 + SGD + MSE

In [7]:
model_dense161 = models.densenet161(pretrained=True)

num_ftrs = model_dense161.classifier.in_features
model_dense161.classifier = nn.Linear(num_ftrs, 196)
model_dense161 = model_dense161.to(device)

lossFun = nn.NLLLoss()
optimizer = optim.SGD(model_dense161.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

model_dense161, training_losses, training_accs = TrainModel(model_dense161, lossFun, optimizer, lrscheduler, n_epochs=10)

1911.0 8144
Epoch 1, Time Used: 190 s, loss: 0.1099, acc: 23.4651
5640.0 8144
Epoch 2, Time Used: 190 s, loss: 0.0363, acc: 69.2534
6917.0 8144
Epoch 3, Time Used: 190 s, loss: 0.0176, acc: 84.9337
7463.0 8144
Epoch 4, Time Used: 191 s, loss: 0.0097, acc: 91.6380
7770.0 8144
Epoch 5, Time Used: 191 s, loss: 0.0055, acc: 95.4077
7937.0 8144
Epoch 6, Time Used: 191 s, loss: 0.0035, acc: 97.4583
8053.0 8144
Epoch 7, Time Used: 191 s, loss: 0.0019, acc: 98.8826
8091.0 8144
Epoch 8, Time Used: 191 s, loss: 0.0012, acc: 99.3492
8102.0 8144
Epoch 9, Time Used: 191 s, loss: 0.0010, acc: 99.4843
8114.0 8144
Epoch 10, Time Used: 191 s, loss: 0.0009, acc: 99.6316
Training Finished


In [8]:
model_dense161.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [9]:
EvaluateModel(model_dense161)

The accuracy on the test dataset: 91 %


91.6925755503047

In [11]:
torch.save(model_dense161.state_dict(), './model_dense161_NLLLoss')

In [12]:
model_res152 = models.resnet152(pretrained=True)
num_ftrs = model_res152.fc.in_features
model_res152.fc = nn.Linear(num_ftrs, 196)
model_res152 = model_res152.to(device)

lossFun = nn.NLLLoss()
optimizer = optim.SGD(model_dense161.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)
model_res152, training_losses, training_accs = TrainModel(model_res152, lossFun, optimizer, lrscheduler, n_epochs=10)

46.0 8144
Epoch 1, Time Used: 195 s, loss: 0.1666, acc: 0.5648
48.0 8144
Epoch 2, Time Used: 196 s, loss: 0.1666, acc: 0.5894
39.0 8144
Epoch 3, Time Used: 195 s, loss: 0.1667, acc: 0.4789
45.0 8144
Epoch 4, Time Used: 196 s, loss: 0.1666, acc: 0.5526
46.0 8144
Epoch 5, Time Used: 195 s, loss: 0.1666, acc: 0.5648
44.0 8144
Epoch 6, Time Used: 195 s, loss: 0.1666, acc: 0.5403
51.0 8144
Epoch 7, Time Used: 195 s, loss: 0.1666, acc: 0.6262
44.0 8144
Epoch 8, Time Used: 196 s, loss: 0.1666, acc: 0.5403
51.0 8144
Epoch 9, Time Used: 195 s, loss: 0.1666, acc: 0.6262
49.0 8144
Epoch 10, Time Used: 196 s, loss: 0.1666, acc: 0.6017
Training Finished
